## Step 1: Setup

Import libraries and understand the NNLS unmixing approach.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

from foodspec.chemometrics.mixture import run_mixture_analysis_workflow

print("✓ All imports successful")
print("\nMixture Analysis Method: Non-Negative Least Squares (NNLS)")
print("  - Decomposes mixture = α₁*S₁ + α₂*S₂ + ... + noise")
print("  - α = fraction of each pure component")
print("  - Constraint: 0 ≤ α ≤ 1 (no negative fractions allowed)")

## Step 2: Generate Synthetic Pure Components and Mixtures

Create two pure-component spectra and then generate mixtures with known fractions.

In [ ]:
# Synthetic pure-component spectra (Gaussian-like peaks)
print("Creating synthetic pure-component spectra...\n")

rng = np.random.default_rng(42)  # Reproducibility
wavenumbers = np.linspace(600, 1800, 120)

# Component 1: peak centered at 1000 cm⁻¹
s1 = np.exp(-0.5 * ((wavenumbers - 1000) / 12) ** 2)

# Component 2: peak centered at 1400 cm⁻¹
s2 = np.exp(-0.5 * ((wavenumbers - 1400) / 10) ** 2)

print(f"Pure Component 1: peak at 1000 cm⁻¹, max intensity = {s1.max():.3f}")
print(f"Pure Component 2: peak at 1400 cm⁻¹, max intensity = {s2.max():.3f}")

# Create mixture spectra with known fractions
print("\nGenerating mixtures with known fractions...")
coeffs_true = []
mixtures = []

for frac in np.linspace(0, 1, 6):  # 0%, 20%, 40%, 60%, 80%, 100%
    c1 = frac
    c2 = 1 - frac

    # Linear mixture with Gaussian noise
    spectrum = c1 * s1 + c2 * s2 + rng.normal(0, 0.01, size=wavenumbers.shape)
    mixtures.append(spectrum)
    coeffs_true.append((c1, c2))

    print(f"  Mixture {len(mixtures)}: C1={c1:.0%}, C2={c2:.0%}")

mixtures = np.vstack(mixtures)
pure = np.vstack([s1, s2])
coeffs_true = np.array(coeffs_true)

print(f"\nGenerated {len(mixtures)} mixtures (rows) × {len(wavenumbers)} wavenumbers (cols)")

## Step 3: Visualize Pure Components and Mixtures

Plot to show how mixtures are combinations of pure spectra.

In [ ]:
# Plot pure components
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

# Pure spectra
axes[0].plot(wavenumbers, s1, label="Pure C1 (1000 cm⁻¹)", linewidth=2)
axes[0].plot(wavenumbers, s2, label="Pure C2 (1400 cm⁻¹)", linewidth=2)
axes[0].set_xlabel("Wavenumber (cm⁻¹)", fontsize=11)
axes[0].set_ylabel("Intensity", fontsize=11)
axes[0].set_title("Pure Components", fontsize=12, fontweight='bold')
axes[0].legend()
axes[0].grid(alpha=0.3)

# Sample mixtures
colors = plt.cm.viridis(np.linspace(0, 1, len(mixtures)))
for i, (mixture, frac) in enumerate(zip(mixtures, coeffs_true)):
    axes[1].plot(wavenumbers, mixture, label=f"{frac[0]:.0%} C1",
                color=colors[i], linewidth=1.5, alpha=0.8)
axes[1].set_xlabel("Wavenumber (cm⁻¹)", fontsize=11)
axes[1].set_ylabel("Intensity", fontsize=11)
axes[1].set_title("Mixture Spectra (varying C1 fraction)", fontsize=12, fontweight='bold')
axes[1].legend(fontsize=9)
axes[1].grid(alpha=0.3)

# Fraction space
for i, (frac, color) in enumerate(zip(coeffs_true, colors)):
    axes[2].scatter(frac[0], frac[1], s=150, color=color, edgecolor='black', linewidth=1.5)
axes[2].plot([0, 1], [1, 0], 'k--', alpha=0.5, label="Constraint: α₁ + α₂ = 1")
axes[2].set_xlabel("Fraction of C1", fontsize=11)
axes[2].set_ylabel("Fraction of C2", fontsize=11)
axes[2].set_title("True Fractions (Mixture Design)", fontsize=12, fontweight='bold')
axes[2].set_xlim(-0.1, 1.1)
axes[2].set_ylim(-0.1, 1.1)
axes[2].legend()
axes[2].grid(alpha=0.3)

plt.tight_layout()
plt.show()

## Step 4: Run NNLS Unmixing

Decompose mixture spectra to estimate component fractions.

In [ ]:
print("Running NNLS unmixing...\n")
res = run_mixture_analysis_workflow(mixtures=mixtures, pure_spectra=pure, mode="nnls")

coeffs_est = res["coefficients"]
residuals = res["residual_norms"]

print("="*70)
print("UNMIXING RESULTS")
print("="*70)
print("\nEstimated Fractions:")
print("   C1_true  C1_est  |Error|  C2_true  C2_est  |Error|  Residual")
print("-" * 70)
for i in range(len(coeffs_true)):
    err_c1 = abs(coeffs_true[i, 0] - coeffs_est[i, 0])
    err_c2 = abs(coeffs_true[i, 1] - coeffs_est[i, 1])
    print(f"{coeffs_true[i, 0]:8.2%} {coeffs_est[i, 0]:7.2%} {err_c1:7.2%}  " +
          f"{coeffs_true[i, 1]:8.2%} {coeffs_est[i, 1]:7.2%} {err_c2:7.2%}  {residuals[i]:8.4f}")

print(f"\nMean Absolute Error: {np.mean(np.abs(coeffs_true - coeffs_est)):.4f}")
print(f"Mean Residual Norm: {np.mean(residuals):.4f}")

## Step 5: Compare True vs. Estimated Fractions

Visualize how well NNLS recovered the known mixture compositions.

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

# Scatter: true vs estimated fractions
axes[0].scatter(coeffs_true[:, 0], coeffs_est[:, 0], s=100, alpha=0.6,
                 edgecolor='black', linewidth=1.5, label="C1")
axes[0].plot([0, 1], [0, 1], 'k--', alpha=0.5, linewidth=2, label="Perfect recovery")
axes[0].set_xlabel("True Fraction", fontsize=11)
axes[0].set_ylabel("Estimated Fraction", fontsize=11)
axes[0].set_title("Component 1: True vs. Estimated", fontsize=12, fontweight='bold')
axes[0].set_xlim(-0.05, 1.05)
axes[0].set_ylim(-0.05, 1.05)
axes[0].legend()
axes[0].grid(alpha=0.3)

# Bar chart: errors
fracs_str = [f"{f:.0%}" for f in coeffs_true[:, 0]]
errors = np.abs(coeffs_true[:, 0] - coeffs_est[:, 0])
colors = plt.cm.RdYlGn_r(errors / errors.max())
axes[1].bar(fracs_str, errors, color=colors, edgecolor='black', linewidth=1.5)
axes[1].set_xlabel("True C1 Fraction", fontsize=11)
axes[1].set_ylabel("Absolute Error", fontsize=11)
axes[1].set_title("Unmixing Error by Composition", fontsize=12, fontweight='bold')
axes[1].grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

print("\nInterpretation:")
print("  - Points near diagonal → good recovery")
print("  - Scatter off diagonal → systematic bias or model mismatch")
print("  - Low bars → accurate unmixing")
print("  - High bars → may indicate spectral interference")

## Step 6: Key Takeaways

### What We Learned:

1. **Linear Unmixing:** Mixture spectra can be modeled as linear combinations of pure spectra
2. **NNLS Constraint:** Non-negativity ensures physically meaningful fractions (0–100%)
3. **Accuracy:** NNLS accurately recovers fractions when:
   - Pure component spectra are available
   - Linear mixing model holds (e.g., no matrix effects)
   - Components have distinct peaks
4. **Residuals:** Residual norms indicate fit quality; large residuals suggest model violation
5. **Practical Limits:** Requires reference materials (pure spectra) in advance

### Real-World Applications:

- **Adulteration Detection:** Identify mineral oil or low-grade oils in blends
- **Ingredient Traceability:** Track origin oils in multi-source blends
- **Process Monitoring:** Track conversion in fermentation/synthesis
- **Quality Certification:** Verify labeled composition vs. actual

### Assumptions & Limitations:

- ✓ **Assumes:** Linear mixing (spectrum = Σ fraction_i * pure_i)
- ✗ **Fails with:** Nonlinear interactions, chemical reactions, matrix effects
- ✓ **Requires:** High-quality pure reference spectra
- ✗ **Cannot:** Discover unknown components (need at least 2 pure spectra for each component)

### Next Steps:

- Test with real oils: load from CSV and unmix
- Try >2 components (e.g., 3-way oil blends)
- Add spectral noise and assess robustness
- Compare NNLS to other unmixing methods (MCR, PCA-based)